# extract FITS image for LSSTCam   

In [34]:
from lsst.daf.butler import Butler
import pathlib


In [31]:
repo = 'embargo'
instrument = 'LSSTCam'
datasetType = 'post_isr_image'
#datasetType = 'preliminary_visit_image'  # background subtracted calibrated image that detection has run on

collections = ["LSSTCam/runs/nightlyValidation", "LSSTCam/defaults",]

butler = Butler(repo, collections=collections, instrument=instrument)



#uncomment to get the help page
#help(Butler) 

## Find the exposures

BLOCK-T519 is the LSSTCam Local Meridian Observations at 80 deg elevation on day = 20250524

BLOCK-T517 — LSSTCam “Sidereal drive off tests” with and without sidereal tracking on day = 20250522

May 3 had over 500 science quality images of M49, Prawn, Trifid-Lagoon,  day_obs = 20250503, 
BLOCK-T365 denotes sciene image taking .


In [32]:

day = 20250522
day2 = 20250524

block = 'BLOCK-T519'  
datasetType = 'post_isr_image'

result = butler.query_datasets( datasetType, limit=None, 
        where = f"(day_obs = {day} OR day_obs = {day2})   AND exposure.science_program='{block}' "

                             )
meridianresults = [r for r in result]

print (f"got {len(meridianresults)} datsets")


got 30316 datsets


In [36]:
#  just wrting out 2 .. 
path = ".."
butler.retrieveArtifacts(destination = path, refs=[meridianresults[2],meridianresults[3]])

[ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000237/post_isr_image_LSSTCam_r_57_MC_O_20250522_000237_R01_S00_LSSTCam_runs_nightlyValidation_19.fits"),
 ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000238/post_isr_image_LSSTCam_r_57_MC_O_20250522_000238_R01_S00_LSSTCam_runs_nightlyValidation_19.fits")]

In [40]:

day = 20250522
block = 'BLOCK-T517'  

result = butler.query_datasets( datasetType, limit=None,  where = 
          f"day_obs = {day}   AND exposure.science_program='{block}' "

                             )
nonsiderealresult = [r for r in result]

print (f"got {len(nonsiderealresult)} images")

got 14880 images


In [41]:
#  just wrting out 2 .. 
print (nonsiderealresult[2])
path = ".."
butler.retrieveArtifacts(destination = path, refs=[nonsiderealresult[2],nonsiderealresult[3]])

post_isr_image@{instrument: 'LSSTCam', detector: 0, exposure: 2025052200140, band: 'r', day_obs: 20250522, group: '2025-05-23T00:28:42.492', physical_filter: 'r_57'} [sc=Exposure] (run=LSSTCam/runs/nightlyValidation/19 id=12dba2f8-01db-4679-94d0-2ae43471908e)


[ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000140/post_isr_image_LSSTCam_r_57_MC_O_20250522_000140_R01_S00_LSSTCam_runs_nightlyValidation_19.fits"),
 ResourcePath("file:///home/o/omullan/notebooks/LSSTCam/runs/nightlyValidation/19/post_isr_image/20250522/MC_O_20250522_000141/post_isr_image_LSSTCam_r_57_MC_O_20250522_000141_R01_S00_LSSTCam_runs_nightlyValidation_19.fits")]

In [49]:

day = 20250503
target = 'M49'  # change to Prawn etc. to get others

reult = butler.query_datasets( datasetType, limit=None,  where = 
            f"day_obs = {day} AND exposure.target_name='{target}'  "
                             )
results = [r for r in result]

print (f"got {len(results)} images")

got 14880 images


In [50]:
#  just wrting out 2 .. 
path = ".."
butler.retrieveArtifacts(destination = path, refs=[results[2],results[3]])

  + Exception Group Traceback (most recent call last):
  |   File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/share/eups/Linux64/daf_butler/g7096c7a84b+2e8246025c/python/lsst/daf/butler/datastores/fileDatastore.py", line 2165, in retrieveArtifacts
  |     ResourcePath.mtransfer(transfer, tuple(to_transfer.items()), overwrite=overwrite)
  |   File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/share/eups/Linux64/resources/g4a157353b6+d65b3c2b70/python/lsst/resources/_resourcePath.py", line 1004, in mtransfer
  |     raise ExceptionGroup(
  | ExceptionGroup: Errors transferring 2 artifacts (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/share/eups/Linux64/resources/g4a157353b6+d65b3c2b70/python/lsst/resources/_resourcePath.py", line 995, in mtransfer
    |     future.result()
    |   File "/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0

LSSTCam has 202 images per exposure - this will list the exposures. 
The last digits after the daynumber  match the 'Seq. No.' in RubinTV

In [19]:
exp=""
exposures = []
for r in nonsiderealresult:
   if r['exposure'] != exp:
        exp=  r['exposure']
        exposures.append(exp)

print (f"there are {len(exposures)} exposures")
print (exposures)


there are 163 exposures
[2025052200226, 2025052200236, 2025052200237, 2025052200238, 2025052200239, 2025052200240, 2025052200241, 2025052200242, 2025052200243, 2025052200244, 2025052200245, 2025052200246, 2025052200247, 2025052200248, 2025052200249, 2025052200250, 2025052200251, 2025052200252, 2025052200253, 2025052200254, 2025052200255, 2025052200256, 2025052200257, 2025052200258, 2025052200259, 2025052200260, 2025052200261, 2025052200262, 2025052200263, 2025052200264, 2025052200265, 2025052200266, 2025052200267, 2025052200268, 2025052200269, 2025052200270, 2025052200271, 2025052200272, 2025052200273, 2025052200274, 2025052200275, 2025052200276, 2025052200283, 2025052200284, 2025052200285, 2025052200286, 2025052200287, 2025052200288, 2025052200289, 2025052200290, 2025052200291, 2025052200292, 2025052200293, 2025052200294, 2025052200295, 2025052200296, 2025052200297, 2025052200298, 2025052200299, 2025052200300, 2025052200301, 2025052200302, 2025052200303, 2025052200304, 2025052200305, 

This is just getting the datasetref for an exposure (this is LSSTcam so that should be 189 images one per detector) unless the corner rafts show up

In [ ]:
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[100]}')


## Write fits 
Write out fits files for the first count images. 
By default you will not have enough space to write all the files.

In [ ]:
images = []
count = 0 
stop = 2  # just write this many images
for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get(datasetType,exp.dataId)
    img.writeFits(fn)
    images.append(exp)
    if count >= stop -1:
        break
    
print (f"{count+1} images output")

Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [ ]:
import glob
filelist = glob.glob('Rubin-*.fits')
print (filelist)

## If you want to plot one of these 

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

for exp in datasetRefs:
    dataId=exp.dataId
    img = butler.get('raw', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()